In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import requests
from bs4 import BeautifulSoup
import re
import json
import unicodedata


# Scrap des activités des anciens sites industriels CASIAS

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/58.0.3029.110 Safari/537.3'}

In [ ]:
def scrap_activites(url_bdd_casias, headers):

    """
    Fonction pour scrapper les codes activités NAF 2008 des sites répertoriés dans CASIAS. 
    La fonction prend en entrée l'url d'un seul site. 
    Elle retourne la liste des codes NAF 2008 alphanumérique pour ce site. 
    
    """

    url_erreurs = []
    
    try : 
        # Récupérer l'information 
        response = requests.get(url_bdd_casias, headers=headers)   

        # Ensuite j'utilise BeautifulSoup pour récupérer les éléments HTML (parser)
        soupe = BeautifulSoup(response.content, 'html.parser')

        # Trouver la section "Activités de l'établissement"
        activites_section = soupe.find('h3', string="3.- Activités de l'établissement")
        # Si elle existe
        if activites_section:
            # Trouver le tableau dans la section "Activités de l'établissement"
            activites_table = activites_section.find_next('table', class_='data')

            if activites_table:
                # Initialiser une liste pour stocker les données des activités secondaires
                activites_secondaires = []

                # Parcourir les lignes du tableau
                for row in activites_table.find_all('tr'):
                    for cell in row.find_all('td'):
                        # Ajouter le texte de la cellule à la liste des activités secondaires
                        activites_secondaires.append(cell.get_text(strip=True))    

            else:
                print("Aucun tableau trouvé dans la section 'Activités de l'établissement'")
        else:
            print("Aucune section 'Activités de l'établissement' trouvée dans le document HTML")
        
        # Définition de la RegEx pour récupérer uniquement le code d'activité
        regex_tiret = "(.+?)\s-\s"

        # Amélioration
        liste_propre = []

        for element in activites_secondaires:
            liste_propre.extend(re.findall(regex_tiret,element))

        return liste_propre

    except requests.exceptions.RequestException as e:
        # Gérer les erreurs liées aux requêtes HTTP
        print("Une erreur s'est produite lors de la récupération de la page :", e)
        url_erreurs.append(url_bdd_casias)  # Ajouter l'URL ayant rencontré une erreur à la liste
        return []
    except Exception as e:
        # Gérer toutes les autres erreurs
        print("Une erreur s'est produite :", e)
        url_erreurs.append(url_bdd_casias)  # Ajouter l'URL ayant rencontré une erreur à la liste
        return []

# Prétraitements

In [3]:
def nettoyer_nom_colonne(texte):
    # Supprimer les accents
    texte_sans_accents = ''.join(char for char in unicodedata.normalize('NFD', texte) if unicodedata.category(char) != 'Mn')
    # Remplacer la ponctuation par des espaces
    texte_sans_ponctuation = re.sub(r'[^\w\s]', ' ', texte_sans_accents)
    # Mettre en minuscules
    texte_minuscules = texte_sans_ponctuation.lower()
    # Remplacer les espaces par des underscores
    texte_avec_underscores = re.sub(r'\s+', '_', texte_minuscules)
    return texte_avec_underscores

In [ ]:
def pretraitements_df_pollution_BRGM(input_bdd_polluants):
   
    df_pollution = pd.read_csv(input_bdd_polluants, sep=';')
    
    # Nettoyage pollution 
    df_pollution = df_pollution.fillna(0)
    df_pollution['Gaz du sol'] = df_pollution['Gaz du sol'].map(lambda x: x == "X")
    df_pollution['Sol'] = df_pollution['Sol'].map(lambda x: x == "X")
    df_pollution['eaux souterraines'] = df_pollution['eaux souterraines'].map(lambda x: x == "X")
    
    # Nettoyage noms colonnes
    df_pollution.rename(columns = lambda x: nettoyer_nom_colonne(x), inplace= True)

    return df_pollution

In [ ]:
def pretraitement_df_CASIAS(input_bdd_casias):
    """
    Prétraitement de la BDD CASIAS

    Suppression des sites non géolocalisés dès le départ afin d'alléger le scrapping. On conserve les doublons de points. 

    Entrée : lien vers la BDD en local
    Sortie : dataframe propre données CASIAS

    """
    df_casias_IDF = pd.read_csv(input_bdd_casias, sep=';')
    df_casias_IDF = df_casias_IDF.fillna(0)
    df_casias_IDF['code_postal'] = df_casias_IDF['code_postal'].astype(int)
    regex_ponctuation = r'[.,;:?!«»(){}\[\]\'"\-…]'
    df_casias_IDF['nom_etablissement'] = df_casias_IDF['nom_etablissement'].str.replace(regex_ponctuation, '')
    df_casias_IDF = df_casias_IDF.fillna(0)
    df_casias_IDF = df_casias_IDF.loc[df_casias_IDF['nature_localisation'] != "Site non gélocalisé"]

    return df_casias_IDF

# Hypothèses de pollution
Croisement des BDD CASIAS et ActiviPol v3 pour obtenir les hypothèses de pollution associé à chaque site industriel. 

In [ ]:
def hypotheses_pollution_usine(liste_activites_usine, df_pollution):

    """
    Fonction pour récupérer les hypothèses de pollution associées à un code NAF 2008. 
    On cherche une note unique de pollution pour chaque groupe de substances.
    Si un site a plusieurs activités, on garde l'hypothèse de pollution la plus élevée. 

        Entrées : 
            liste des codes NAF 2008 d'un site CASIAS 
            dataframe BRGM avec les hypothèses de pollution des groupes de substances

        Sortie : 
            pour 1 site CASIAS
            dictionnaire avec les hypothèses de pollution par activités, globales et les indicateurs agrégés
    """
      
    # Extraction des polluants associés à chaque activité dans le df de corrélation BRGM
    dict_NAF2008_Pol = {}
        
    for code in liste_activites_usine:
            try : 
                # Extraction des notes associées à l'activité
                contenu = df_pollution[code].tolist()
                
                # Stockage dans un dictionnaire
                dict_NAF2008_Pol[code] = {"values": contenu}

            except KeyError:
            # Gérer l'erreur lorsque la colonne correspondant au code NAF 2008 n'est pas trouvée dans le DataFrame df_pollution
                print(f"La colonne correspondant au code {code} n'existe pas dans le DataFrame df_pollution.")
        
            except Exception as e:
            # Gérer toutes les autres exceptions
                 print(f"Une erreur s'est produite : {e}")

    # On ne conserve que la note la plus haute pour chaque substance (regroupement au niveau de l'usine)
    resultats_filtres = {}

    # Parcourir chaque identifiant dans le dictionnaire de résultats
    for identifiant, valeurs in dict_NAF2008_Pol.items():
        # Si c'est le premier identifiant, copier simplement les valeurs
        if not resultats_filtres:
            resultats_filtres = valeurs
        else:
            # Parcourir chaque clé de colonne dans les valeurs du résultat
            for colonne, valeur in valeurs.items():
                # Sélectionner la valeur la plus grande entre la valeur actuelle et la valeur précédente
                resultats_filtres[colonne] = max(resultats_filtres[colonne], valeur)
                
    # Ajout des valeurs simplifiées dans le dictionnaire de résultats 
    dict_NAF2008_Pol['valeurs_new'] = resultats_filtres['values']

    # Compter le nombre total de substances potentiellement présentes sur le site 
    nb_substances = len([nombre for nombre in dict_NAF2008_Pol.get('valeurs_new') if nombre != 0])
    dict_NAF2008_Pol['nb_substances'] = nb_substances
    
    # Note moyenne pollution totale
    valeurs_non_nulles = [nombre for nombre in dict_NAF2008_Pol.get('valeurs_new') if nombre != 0]
    dict_NAF2008_Pol['m_totale'] = sum(valeurs_non_nulles) / nb_substances

    # Note moyenne produits chimiques 
    try:
        # Tentative de calcul de la moyenne
        dict_NAF2008_Pol["m_chimique"] = sum([nombre for nombre in dict_NAF2008_Pol['valeurs_new'][0:3] if nombre != 0])/len([nombre for nombre in dict_NAF2008_Pol['valeurs_new'][0:3] if nombre != 0])
    except ZeroDivisionError:
        dict_NAF2008_Pol["m_chimique"] = 0

    try : 
        dict_NAF2008_Pol['m_elem_mineraux'] = sum([nombre for nombre in dict_NAF2008_Pol['valeurs_new'][3:8] if nombre != 0])/len([nombre for nombre in dict_NAF2008_Pol['valeurs_new'][3:8] if nombre != 0])
    except ZeroDivisionError:
        dict_NAF2008_Pol["m_elem_mineraux"] = 0
        
    try :
        dict_NAF2008_Pol['m_ETM'] = sum([nombre for nombre in dict_NAF2008_Pol['valeurs_new'][8:23] if nombre != 0])/len([nombre for nombre in dict_NAF2008_Pol['valeurs_new'][8:23] if nombre != 0])
    except ZeroDivisionError:
        dict_NAF2008_Pol["m_ETM"] = 0

    try :
        dict_NAF2008_Pol['m_organiques'] = sum([nombre for nombre in dict_NAF2008_Pol['valeurs_new'][23:55] if nombre != 0])/len([nombre for nombre in dict_NAF2008_Pol['valeurs_new'][23:55] if nombre != 0])
    except ZeroDivisionError:
        dict_NAF2008_Pol["m_organiques"] = 0  
    
    try:
        dict_NAF2008_Pol['m_pharmaceutiques'] = sum([nombre for nombre in dict_NAF2008_Pol['valeurs_new'][55:64] if nombre != 0])/len([nombre for nombre in dict_NAF2008_Pol['valeurs_new'][55:64] if nombre != 0])
    except ZeroDivisionError:
        dict_NAF2008_Pol["m_pharmaceutiques"] = 0
    
    try: 
        dict_NAF2008_Pol['m_phytosanitaires'] = sum([nombre for nombre in dict_NAF2008_Pol['valeurs_new'][64:73] if nombre != 0])/len([nombre for nombre in dict_NAF2008_Pol['valeurs_new'][64:73] if nombre != 0])
    except ZeroDivisionError:
        dict_NAF2008_Pol["m_phytosanitaires"] = 0
        
    return dict_NAF2008_Pol

In [ ]:
def boucle_hyp_pollution_all_casias(df_casias, df_pollution, headers): 
    """
    Fonction pour scrapper les activités, et récupérer les hypothèses de pollution correspondates sur toutes la BDD Casias.

    Entrée : 
        BDD Casias
        BDD BRGM pollution
        headers scrapping
        
    Sortie :  dictionnaire avec les données pour toute la BDD Casias 
    """

    dict_final_hyp_pollution = {}

    # Pour chaque entrée de la BDD CASIAS (anciens sites industriels)
    for index, row in df_casias.iterrows():
        code_usine = row['code_metier']
        
        # Récupération des codes activités NAF 2008 via la fiche de site en ligne 
        # sortie : liste des codes alphanumériques NAF 2008 associés
        usine_url = row['fiche_risque']
        activites_usine = scrap_activites(usine_url, headers)

        if activites_usine:
            # Extraction des hypothèses de pollution du site 
            # sortie : dictionnaire activité / pollution 
            try:
                dict_hyp_pollution_usine = hypotheses_pollution_usine(activites_usine, df_pollution)
                dict_final_hyp_pollution[code_usine] = dict_hyp_pollution_usine
            
            except KeyError:
                # Gérer l'erreur lorsque la colonne correspondant au code NAF 2008 n'est pas trouvée dans le DataFrame df_pollution
                print(f"L'un des codes activités n'a pas été trouvé le DataFrame df_pollution.")
            
            except Exception as e:
                # Gérer toutes les autres exceptions
                print(f"Une erreur s'est produite : {e}")
                
        
        else: 
            pass

        
    return dict_final_hyp_pollution

In [ ]:
def merge_results_vers_df_pollution(dict_final_hyp_pollution, df_casias, df_pollution):

    """
    Fonction pour ajouter les hypothèses de pollution au dataframe Casias initial. 

    """

    # Valeurs max par substances
    df_SSP_pollution = pd.DataFrame([value['valeurs_new'] for value in dict_final_hyp_pollution.values()])

    # Colonnes
    sous_groupes = list(df_pollution['Sous-groupe de substances'])
    df_SSP_pollution.columns = sous_groupes

    # ID 
    df_SSP_pollution['code_metier'] = dict_final_hyp_pollution.keys()

    # Indices de pollution aggrégés 
    df_SSP_pollution['m_totale'] = [value['m_totale'] for value in dict_final_hyp_pollution.values()]
    df_SSP_pollution['nb_substances'] = [value['nb_substances'] for value in dict_final_hyp_pollution.values()]
    df_SSP_pollution['m_chimique'] = [value['m_chimique'] for value in dict_final_hyp_pollution.values()]
    df_SSP_pollution['m_elem_mineraux'] = [value['m_elem_mineraux'] for value in dict_final_hyp_pollution.values()]
    df_SSP_pollution['m_ETM'] = [value['m_ETM'] for value in dict_final_hyp_pollution.values()]
    df_SSP_pollution['m_organiques'] = [value['m_organiques'] for value in dict_final_hyp_pollution.values()]
    df_SSP_pollution['m_pharmaceutiques'] = [value['m_pharmaceutiques'] for value in dict_final_hyp_pollution.values()]
    df_SSP_pollution['m_phytosanitaires'] = [value['m_phytosanitaires'] for value in dict_final_hyp_pollution.values()]
    
    # Nouveau df CASIAS avec les valeurs de pollution par usine 
    df_casias_pollution = pd.merge(df_casias, df_SSP_pollution, how='outer', on='code_metier')
   
    return df_casias_pollution

# Fonction main

In [ ]:
# Utilisation finale de toutes les fonctions

# Charger les datasets 
url_bdd_casias_IDF = "/Users/charlotteberthier/Documents/Memoire/9_Git_heritages_toxiques/source/CASIAS/240422_CASIAS_IDF.csv"
url_bdd_casias_93 = "/Users/charlotteberthier/Documents/Memoire/9_Git_heritages_toxiques/source/CASIAS/240424_CASIAS_93.csv"
url_bdd_polluants = "/Users/charlotteberthier/Documents/Mémoire/Production/EDV_Exploration_des_données/Data intermédiaire/Corr_sousgrpe_polluants_activités.csv"

df_pollution = pretraitements_df_pollution_BRGM(url_bdd_polluants)
df_casias = pretraitement_df_CASIAS(url_bdd_casias_IDF)

# Boucle sur tout CASIAS 
dict_final_hyp_pollution = boucle_hyp_pollution_all_casias(df_casias, df_pollution, headers)

# Conversion du dictionnaire de résultats vers CASIAS 

df_final_casias_pollution = merge_results_vers_df_pollution(dict_final_hyp_pollution, df_casias, df_pollution)

# Sauvegarde des données

In [ ]:
df_final_casias_pollution.to_csv('df_final_casias_pollution.csv', index=False)